In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
from campfin_data.utils import *

pdc_map = SOSPDCMap.objects.filter(sos_ballot_id=999992)
committees = list(set([pm.pdc_filer_id for pm in pdc_map]))
cmte_objs = ScrapedCommittee.objects.filter(filer_id__in=committees)
donations = ScrapedContribution.objects.filter(filing_committee__in=cmte_objs)
top_donors = account_for_refunds(
    donations.values('donor', 'city', 'state', 'employer', 'occupation').annotate(
    sum=Sum('amount'), num=Count('donor')).order_by('-sum', '-num', 'donor'), cmte_objs)
print top_donors[0]

{'city': u'SEATTLE', 'sum': Decimal('358000.000'), 'employer': u'', 'state': u'WA', 'num': 6, 'donor': u'FAMILIES FOR SUSTAINABLE TRANSIT', 'occupation': u''}


In [3]:
#candidates = get_candidates_by_district()

In [4]:
election = BallotElection.objects.get(is_current=True)
district = BallotDistrict.objects.get(district_id=80753)
candidates = BallotRaceSummary.objects.filter(district=district)
print candidates
races = BallotRaceSummary.objects.filter(race_id=999999)
for race in races:
    if not race.district:
        break
        #race.district = district
        #race.save()

[<BallotRaceSummary: Yes (Proposition 1 (Sound Transit 3) - Light rail, commuter rail, and bus service expansion, SOUND TRANSIT)>, <BallotRaceSummary: No (Proposition 1 (Sound Transit 3) - Light rail, commuter rail, and bus service expansion, SOUND TRANSIT)>]
